# VacationPy
---

## Importing Libraries and Loading the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Importing API key
from api_keys import geoapify_key

In [5]:
# Loading the CSV file created in WeatherPy into a Pandas DataFrame
city_data_df = pd.read_csv("Output_Data/cities.csv")

# Displaying a sample of the data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chokwe,-24.5333,32.9833,23.15,89,21,4.77,MZ,1675717707
1,1,hobart,-42.8794,147.3294,15.03,72,20,1.03,AU,1675717652
2,2,atuona,-9.8000,-139.0333,25.61,76,5,6.54,PF,1675717707
3,3,nevel,56.0209,29.9284,-1.73,98,100,2.26,RU,1675717708
4,4,avarua,-21.2078,-159.7750,29.03,84,97,4.12,CK,1675717708


---

### Step 1: Creating a map that displays a point for every city in the `city_data_df` DataFrame. The size of the points correspond to the humidity in each city.

In [16]:
# Configuring map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    alpha = 0.5,
    color = "City"
)

# Displaying map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrowing down the `city_data_df` DataFrame to find ideal weather conditions

In [29]:
# Narrowing down to cities that fit criteria
shortlist_df = city_data_df.loc[(city_data_df['Max Temp'] < 25) & 
                                (city_data_df['Max Temp'] > 15) & 
                                (city_data_df['Humidity'] < 70) &
                                (city_data_df['Humidity'] > 40) &
                                (city_data_df['Cloudiness'] < 20)]

# Dropping any rows with null values
clean_shortlist_df = shortlist_df.dropna()

# Displaying the data
clean_shortlist_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,pacific grove,36.6177,-121.9166,15.23,63,0,3.60,US,1675717519
75,75,coihaique,-45.5752,-72.0662,18.97,42,0,4.63,CL,1675717729
111,111,marsh harbour,26.5412,-77.0636,22.62,66,0,7.33,BS,1675717739
114,114,salalah,17.0151,54.0924,22.05,56,0,2.06,OM,1675717740
165,165,mar del plata,-38.0023,-57.5575,22.01,59,0,8.23,AR,1675717739
193,193,ancud,-41.8697,-73.8203,16.96,66,0,2.63,CL,1675717767
273,273,lompoc,34.6391,-120.4579,18.21,63,0,5.81,US,1675717791
285,285,laguna,38.4210,-121.4238,16.16,46,0,7.72,US,1675717795
310,310,kaitangata,-46.2817,169.8464,19.66,48,13,4.49,NZ,1675717803
343,343,san joaquin,37.9333,-121.3011,15.74,53,0,5.66,US,1675717817


### Step 3: Creating a new DataFrame called `hotel_df`.

In [34]:
# Using Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_shortlist_df.copy()

# Adding empty column, "Hotel Name", to the DataFrame to store hotels found using Geoapify API
hotel_df['Hotel Name'] = ""

# Displaying the data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
54,54,pacific grove,36.6177,-121.9166,15.23,63,0,3.60,US,1675717519,
75,75,coihaique,-45.5752,-72.0662,18.97,42,0,4.63,CL,1675717729,
111,111,marsh harbour,26.5412,-77.0636,22.62,66,0,7.33,BS,1675717739,
114,114,salalah,17.0151,54.0924,22.05,56,0,2.06,OM,1675717740,
165,165,mar del plata,-38.0023,-57.5575,22.01,59,0,8.23,AR,1675717739,
193,193,ancud,-41.8697,-73.8203,16.96,66,0,2.63,CL,1675717767,
273,273,lompoc,34.6391,-120.4579,18.21,63,0,5.81,US,1675717791,
285,285,laguna,38.4210,-121.4238,16.16,46,0,7.72,US,1675717795,
310,310,kaitangata,-46.2817,169.8464,19.66,48,13,4.49,NZ,1675717803,
343,343,san joaquin,37.9333,-121.3011,15.74,53,0,5.66,US,1675717817,


### Step 4: For each city, using the Geoapify API to find the first hotel located within 10,000 metres of the coordinates.

In [35]:
# Setting parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":20
}

# Printing message to follow up the hotel search
print("Starting hotel search")

# Iterating through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # getting latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Adding filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Setting base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Making API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Converting API response to JSON format
    name_address = name_address.json()
    
    # Grabbing first hotel from the results and storing the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, hotel name is set as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Logging the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Displaying the data
hotel_df

Starting hotel search
pacific grove - nearest hotel: Pacific Grove Inn
coihaique - nearest hotel: No hotel found
marsh harbour - nearest hotel: Lofty Fig Villas
salalah - nearest hotel: Muscat International Hotel
mar del plata - nearest hotel: Nuevo Ostende
ancud - nearest hotel: Cabañas
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
laguna - nearest hotel: Holiday Inn Express & Suites
kaitangata - nearest hotel: Hotel South Otago
san joaquin - nearest hotel: I-5 Inn
bubaque - nearest hotel: Chez Julio
salinas - nearest hotel: The Traveler's Hotel
dunedin - nearest hotel: Scenic Hotel Dunedin City
taoru - nearest hotel: No hotel found
srivardhan - nearest hotel: No hotel found
sao filipe - nearest hotel: Bela Vista
waiouru - nearest hotel: No hotel found
tindwari - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
merritt island - nearest hotel: No hotel found
teknaf - nearest hotel: No hotel found
hisar - nearest hotel: OYO Mid City Hotel

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
54,54,pacific grove,36.6177,-121.9166,15.23,63,0,3.60,US,1675717519,Pacific Grove Inn
75,75,coihaique,-45.5752,-72.0662,18.97,42,0,4.63,CL,1675717729,No hotel found
111,111,marsh harbour,26.5412,-77.0636,22.62,66,0,7.33,BS,1675717739,Lofty Fig Villas
114,114,salalah,17.0151,54.0924,22.05,56,0,2.06,OM,1675717740,Muscat International Hotel
165,165,mar del plata,-38.0023,-57.5575,22.01,59,0,8.23,AR,1675717739,Nuevo Ostende
193,193,ancud,-41.8697,-73.8203,16.96,66,0,2.63,CL,1675717767,Cabañas
273,273,lompoc,34.6391,-120.4579,18.21,63,0,5.81,US,1675717791,Embassy Suites by Hilton Lompoc Central Coast
285,285,laguna,38.4210,-121.4238,16.16,46,0,7.72,US,1675717795,Holiday Inn Express & Suites
310,310,kaitangata,-46.2817,169.8464,19.66,48,13,4.49,NZ,1675717803,Hotel South Otago
343,343,san joaquin,37.9333,-121.3011,15.74,53,0,5.66,US,1675717817,I-5 Inn


### Step 5: Adding the hotel name and the country as additional information in the hover message for each city in the map.

In [43]:
# Configuring map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    alpha = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Displaying map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)